# Primo progetto laboratorio Social Computing
### *Costruzione di un sottografo della rete di twitter, utlizzando i moduli di python tweepy e networkx.*

* Tavano Matteo mat. 154255
* Alessandro Dalla Riva mat. 151881
* Propedo Demien mat. 153260
* Scozzai Samuele mat. 154175

Importo le **librerie**:

In [164]:
import os
import json
import pprint
import tweepy
import random
import networkx as nx
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from scipy import stats
from networkx.readwrite import json_graph
import inspect
from collections import *
from networkx.algorithms.approximation import clique
from pyvis.network import Network

Importo le **chiavi** dall'account developer di twitter, utili per il download dei dati provenienti dal social network:

In [165]:
api_key = "ZwRUucJU77qVjVxQgqAhPmaaX"
api_secret = "QzCFP6OTu9uNGj9YhQuYL5lr27EbV8Np38uwllFi6D8GE8PtsE"
access_token = "1588461193928364032-bMRfOj9Xq9nJArAPMmR32naFtcNcNC"
access_secret = "xHjg6oTGM28KGH0dYDCXYu5MktvqkQ5wjHjWwIIyr4qq9"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAL63iwEAAAAAdwdMMdk3pjwk9LCqVp1o2mp5AOM%3DkYs8l09vh6814psR071MTFyhEwD471Axfjheue7prFqoADyxur"

Faccio la **serializzazione di JSON**. 
Al fine di ottenere in locale i dati su cui lavoreremo, sarà necessario definire due funzioni per il salvataggio dei dati in locale e la lettura di questi ultimi:

In [166]:
data_folder = "data"

# Salvataggio dati in locale
def serialize_json(folder, filename, data):
    # Se la cartella non esiste la creo
    if not os.path.exists(folder):
        os.makedirs(folder, exist_ok=True)
    with open(f"{folder}/{filename}", 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
        f.close()
    print(f"Data serialized to path: {folder}/{filename}")

# Lettura file JSON da locale
def read_json(path):
    if os.path.exists(path):
        with open(path, "r", encoding="utf8") as file:
            data = json.load(file)
        print(f"Data read from path: {path}")
        return data
    else:
        print(f"No data found at path: {path}")
        return {}

## 1. Scarichiamo i dati dell'account del profilo di *KevinRoitero*

Come primo passo,  avvalendomi dell'API di twitter *scarico i dati inerenti al profilo* interessato, annettendo ID, username e descrizione di ogni suo follower.
Alla fine di questo processo, i dati vengono *serializzati in locale*.

In [167]:
client = tweepy.Client(bearer_token = bearer_token, wait_on_rate_limit=True) # wait_on_rate_limit attende quando sforo i limiti dell'endpoint

user_to_find = ["KevinRoitero"]
all_followers = {}

for current_user in user_to_find:
    # Cerco l'utente per recuperare i suoi dati
    response = client.get_user(username=current_user)
    user_id = response.data.id
    username = response.data.username
    all_followers[username] = []


    print(f"Looking for {current_user} followers.")
    # Uso paginator perchè non so a priori quanti follower abbiano 
    for current_user_followers in tweepy.Paginator(client.get_users_followers, id = user_id, user_fields = ["description"], max_results = 1000):
        # Non tutti gli oggetti sono serializzabili direttamente come JSON. User non lo è, serve un parsing
        this_user_followers = []
        for follower in current_user_followers.data:
            found_follower = {
                            "id": follower["id"], 
                            "name": follower["name"], 
                            "username": follower["username"],
                            "description": follower["description"],
                        }
            this_user_followers.append(found_follower)
        all_followers[username] += this_user_followers
        print(f"Found {len(all_followers[username])} followers for {username}")


serialize_json(data_folder, "followers.json", all_followers)


Looking for KevinRoitero followers.
Found 135 followers for KevinRoitero
Data serialized to path: data/followers.json


## 2/3. Trovare dettagli dei follower di @KevinRoitero

Una volta ottenuta la serializzazione in locale dei dati dei follower acquisiti, facciamo un ulteriore passo per un'analisi più approfondia di ciascun follower, in particolare:
* **Download follower** di *KevinRoitero*, con attributi di default, descrizione, metriche pubbliche e "protected"
* **Download del numero di tweet** pubblicati nell'ultima settimana
* **Serializzazione di ogni account** con nessuna protezione dell'account, considerando al massimo 1000 followers

In [ ]:
client = tweepy.Client(bearer_token, wait_on_rate_limit=True)
user = client.get_user(username="KevinRoitero")

all_user_follower = client.get_users_followers(
    user.data.id, user_fields=["description", "public_metrics","protected"], max_results=150)

all_followers = {}
for follower in all_user_follower.data:
    query = "from:"+follower["username"]
    response = client.search_recent_tweets(query)
    recent_tweets_count = response.meta["result_count"]
    
    follower_fields = {
        "name": follower["name"],
        "username": follower["username"],
        "description": follower["description"],
        "public_metrics": follower["public_metrics"],
        "protected": follower["protected"],
        "recent_tweets_count": recent_tweets_count
    }

    followers_of_follower = client.get_users_followers(follower["id"], user_fields=["description", "public_metrics","protected"], max_results=1000)
    followers_of_follower_count=follower["public_metrics"]["followers_count"]
    print("i follower di "+str(follower["username"])+" sono "+str(followers_of_follower_count))
    all_followers_of_follower={}
    print("Protetto?"+str(follower["protected"]))

    if followers_of_follower_count>0 and follower["protected"]==False:
            for follower_of_follower in followers_of_follower.data:
                follower_of_follower_fields = {
                    "name": follower_of_follower["name"],
                    "username": follower_of_follower["username"],
                    "description": follower_of_follower["description"],
                    "public_metrics": follower_of_follower["public_metrics"],
                    "protected": follower_of_follower["protected"]
                }
                all_followers_of_follower[follower_of_follower["id"]]=(follower_of_follower_fields)
            follower_fields["followers"]=all_followers_of_follower
   
    all_followers[follower["id"]] = follower_fields

serialize_json(data_folder, "followers_di_KR.json", all_followers)

## 4. NetworkX

Utilizzo la *libreria NetworkX* per costruire la **rete sociale diretta**:

* Creo il grafo diretto dove i **nodi** sono *KevinRoitero* ed i suoi rispettivi follower. L'id del nodo corrisponde all'id dell'profilo utente e ogni nodo ha i seguenti attributi: username, descrizione e numero di follower di quel profilo

In [ ]:
directed_graph = nx.DiGraph()
directed_graph.add_node("KevinRoitero")

for follower in all_user_follower.data:
    directed_graph.add_node(follower["id"], username=follower["username"], description=follower["description"], follower_count=follower["public_metrics"]["followers_count"])
    directed_graph.nodes(data = True)

# Stampo i nodi del grafo diretto
print("Questi sono i nodi:\n",directed_graph.nodes())

* Utilizzo il file JSON prodotto al punto 3 e aggiungo al grafo un **arco** per ciascuna coppia di profili per i quali esiste un arelazione di following

In [ ]:
with open('data/followers.json') as json_file:
    data = json.load(json_file)

with open('data/followers_di_KR.json') as json_file:
    data2 = json.load(json_file)

# Creo gli archi tra Kevin Roitero e i suoi followers
i=0     
while i < len(all_followers[username]):
    id = data['KevinRoitero'][i]['id']
    directed_graph.add_edge("KevinRoitero",id)
    i += 1
 
 
# Creo gli archi tra i followers di Kevin Roitero e i loro followers
i=0
j=0

while i < len(all_followers[username]):    
    id = data['KevinRoitero'][i]['id']
    
    follower_count_id = data2[str(id)]['public_metrics']['followers_count']
    

    while j < min(follower_count_id,1000):
        follower_id = list(data2[str(id)]['followers'].keys())[j]
        directed_graph.add_edge(id,follower_id)
        j += 1
       
    i += 1

# Stampo i dati relativi agli archi 
print("\nQuesti sono gli archi:\n",directed_graph.edges())

## 5. Grafo indiretto e preferential attachment

In questa sezione, oltre a rendere **indiretto** il grafo di prima, utilizzeremo il metodo del **preferential attachment** al fine di ottenere un grafo con il doppio dei nodi e per ognuno di essi, ci saranno 2 archi uscenti.

* Trasformo il grafo diretto in **indiretto**

In [178]:
undirected_graph1 = directed_graph.to_undirected()

* Uso il metodo del **preferential attachment**

In [ ]:
undirected_graph1 = directed_graph.to_undirected()
initial_number_nodes = len(undirected_graph1.nodes)
undirected_graph = nx.barabasi_albert_graph(2*initial_number_nodes, 2, initial_graph=undirected_graph1)
nx.draw(undirected_graph, node_size = 1)

## 6. Visualizzazione dei due grafi

In questo punto, ci occupiamo di ottenere due visualizzazioni, per ognuno dei due grafi, una **interattiva** utilizzando *PyVis* e una **statica** utilizzando *NetworkX*:

* **Visualizzazione interattiva** con *PyVis*

In [179]:
# Dinamica utilizzando Pvyis per il grafo diretto:
nt = Network(
    height = "100%",
    width = "100%",
    bgcolor = "#222222",
    font_color = "white",
    heading = "Directed graph"
)
nt.barnes_hut()
nt.from_nx(directed_graph)
neighbor_map = nt.get_adj_list()
for node in nt.nodes:
    node["value"] = len(neighbor_map[node["id"]])
    
nt.show("html/directed_graph.html")

In [180]:
# Dinamica utilizzando Pvyis per il grafo indiretto:
nt = Network(
    height = "100%",
    width = "100%",
    bgcolor = "#222222",
    font_color = "white",
    heading = "Undirected graph"
)
nt.barnes_hut()
nt.from_nx(undirected_graph)
neighbor_map = nt.get_adj_list()
for node in nt.nodes:
    node["value"] = len(neighbor_map[node["id"]])
    
nt.show("html/undirected_graph.html")

* **Visualizzazione statica** con *NetworkX*

In [ ]:
# statica usando networkx:
# nel undirected uso il grado come peso totale nella rap. grafica
# nel directed, invece uso solamente il in_degree
d = dict(directed_graph.in_degree)
d2= dict(undirected_graph.degree)

print("Visualizzazione statica del primo grafo:\n")
nx.draw(directed_graph, cmap = plt.get_cmap('jet'), node_size=[v * 10 for v in d.values()])

print("\nVisualizzazione statica del secondo grafo:\n")
nx.draw(undirected_graph, node_size=[v * 10 for v in d2.values()])

# utilizziamo il layout di Fruchterman Reingold
nx.draw_networkx(directed_graph, pos=nx.spring_layout(directed_graph), node_size= 1, with_labels=False )
nx.draw_networkx(undirected_graph, pos=nx.spring_layout(undirected_graph), node_size = 100)

## 7. Componente fortemente connessa

Per ognuno dei due grafici, troviamo la **più grande componente fortemente connessa**. La evidenziamo con il colore rosso e il resto dei nodi li coloriamo di nero.

In [ ]:
# la funzione clique.max_clique() non è implementata per i grafi diretti, quindi uso la sua versione indiretta
max_cl = clique.max_clique(undirected_graph1)
clique_subgraph = undirected_graph1.subgraph(max_cl)

#ciclo for per ogni nodo, se appartiene alla cricca max-> rosso else nero
color_map = []
print(clique_subgraph.nodes())

for node in undirected_graph1:
    if node in clique_subgraph.nodes:
        color_map.append('red')
    else: 
        color_map.append('black') 
             
nx.draw(undirected_graph1, node_color=color_map, with_labels=False)
plt.show()
plt.close()

In [ ]:
max_cl2 = clique.max_clique(undirected_graph)
clique_subgraph2 = undirected_graph.subgraph(max_cl2)

#ciclo for per ogni nodo, se appartiene alla cricca max-> rosso else nero
color_map = []
print(clique_subgraph2.nodes())

for node in undirected_graph:
    if node in clique_subgraph2.nodes:
        color_map.append('red')
    else: 
        color_map.append('black') 
             
nx.draw(undirected_graph, node_color=color_map, with_labels=False)
plt.show()
plt.close()

## 8. Distanze

Calcolo le **distanze** sui due grafi, utilizzando le opportune funzioni di *NetworkX*:

* Centro

In [ ]:
# Per il grafo diretto non è possibile calcolare il centro, quindi lo calcolo con la sua versione indiretta

# Grafo 1
center = nx.center(undirected_graph1)
print(center)

# Grafo 2
center2 = nx.center(undirected_graph)
print(center2)

* Raggio

In [182]:
# Grafo 1
radius = nx.radius(undirected_graph1)
print(radius)

# Grafo 2
radius2 = nx.radius(undirected_graph)
print(radius2)

2
4


* Distanza Media

In [185]:
# Grafo 1
average_distance = nx.average_shortest_path_length(undirected_graph1)
print(average_distance)

# Grafo 2
average_distance2 = nx.average_shortest_path_length(undirected_graph)
print(average_distance2)

2.6016395731215485
2.806182980755515


* Distanza Massima

In [ ]:
# Grafo 1
max_distance = nx.diameter(undirected_graph1)
print(max_distance)

# Grafo 2
max_distance2 = nx.diameter(undirected_graph)
print(max_distance2)

## 9. Misure di centralità

Calcolo le seguenti **misure di centralità** sui due grafi:

* Betweenness centrality

In [ ]:
# Grafo 1
betweenness_centrality = nx.betweenness_centrality(directed_graph)
print(betweenness_centrality)

# Grafo 2
betweenness_centrality2 = nx.betweenness_centrality(undirected_graph)
print(betweenness_centrality2)

* Closeness centrality

In [ ]:
# Grafo 1
closeness_centrality = nx.closeness_centrality(directed_graph)
print(closeness_centrality)

# Grafo 2
closeness_centrality2 = nx.closeness_centrality(undirected_graph)
print(closeness_centrality2)

* Degree centrality

In [ ]:
# Grafo 1
degree_centrality = nx.degree_centrality(directed_graph)
print(degree_centrality)

# Grafo 2
degree_centrality2 = nx.degree_centrality(undirected_graph)
print(degree_centrality2)

* In-degree centrality

In [ ]:
# Grafo 1
in_degree_centrality = nx.in_degree_centrality(directed_graph)
print(in_degree_centrality)

# Grafo 2
# Per i grafi indiretti non è possibile calcolare l'in-degree centrality

* Out-degree centrality

In [ ]:
# Grafo 1
out_degree_centrality = nx.out_degree_centrality(directed_graph)
print(out_degree_centrality)

# Grafo 2
# Per i grafi indiretti non è possibile calcolare l'out-degree centrality

* Page Rank

In [ ]:
# Grafo 1
page_rank = nx.pagerank(directed_graph)
print(page_rank)

# Grafo 2
page_rank2 = nx.pagerank(undirected_graph)
print(page_rank2)

* HITS

In [ ]:
# Grafo 1
hits = nx.hits(directed_graph)
print(hits)

# Grafo 2
hits2 = nx.hits(undirected_graph)
print(hits2)

## 10. Coefficienti per stimare la "small-world-ness"

Calcolo il **coefficiente omega** e il **coefficiente sigma** dei due grafi:
1. se omega ha un *valore ≈ 0*, significa che ha le caratteristiche dello small-world
2. se sigma ha un *valore > 1*, il grafo è classificato come small-world 

* Coefficiente omega:

In [ ]:
# Grafo 1
omega = nx.omega(undirected_graph1)
print(omega)

# Grafo 2
omega2 = nx.omega(undirected_graph)
print(omega2)

* Coefficiente sigma:

In [ ]:
# Grafo 1
sigma = nx.sigma(undirected_graph1)
print(sigma)

# Grafo 2
sigma2 = nx.sigma(undirected_graph)
print(sigma2)